# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [8]:
import sys
print(sys.executable)


/Users/Marc/Documents/GitHub/wikipedia-scraper/myenv/bin/python


$ python -m pip install requests

In [1]:
# import the requests library (1 line)

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)

# assign the /status endpoint to another variable called status_url (1 line)

# query the /status endpoint using the get() method and store it in the req variable (1 line)

# check the status_code using a condition and print appropriate messages (4 lines)
import requests  # import the requests library

root_url = "https://country-leaders.onrender.com"  # assign root URL
status_url = f"{root_url}/status"  # assign the /status endpoint

req = requests.get(status_url)  # make the GET request

if req.status_code == 200:
    print(req.text)  # print response text if OK
else:
    print(f"Error: {req.status_code}")  # print status code if not OK


"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [8]:
# Set the countries_url variable (1 line)

# query the /countries endpoint using the get() method and store it in the req variable (1 line)

# Get the JSON content and store it in the countries variable (1 line)

# display the request's status code and the countries variable (1 line)import requests

# Step 1: Set up the base and endpoint URLs
import requests

root_url = "https://country-leaders.onrender.com"
countries_url = f"{root_url}/countries"

# Start a session (to store cookies)
session = requests.Session()

# Pretend to be a real browser
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36"
})

# First, make a GET request to the root to get the cookie
session.get(root_url)  # Now the session has the necessary cookie

# Now access the protected endpoint
response = session.get(countries_url)

# Check result
if response.status_code == 200:
    countries = response.json()
    print(countries)
else:
    print(f"Error: {response.status_code}")
    print(response.text)




Error: 403
{"message":"The cookie is missing"}


Check this out ! https://community.render.com/t/setting-cookies-onrender-com/7886?utm_source=chatgpt.com


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [12]:
# Set the cookie_url variable (1 line)

# Query the enpoint, set the cookies variable and display it (2 lines)import requests

# Set the URL for the cookie endpoint
cookie_url = "https://country-leaders.onrender.com/cookie"

# Query the endpoint
response = requests.get(cookie_url)

# Extract the cookie (it comes as JSON)
cookies = response.json()
print(cookies)


{'message': 'The cookie has been created'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [14]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

# display the countries variable (1 line)
# Query the /countries endpoint using the cookie
import requests

# Step 1: Get the cookie
cookie_url = "https://country-leaders.onrender.com/cookie"
response = requests.get(cookie_url)

# Extract the actual cookie from the response
cookie_jar = response.cookies.get_dict()

# Print cookie name and value
for name, value in cookie_jar.items():
    print(f"Cookie name: {name}")
    print(f"Cookie value: {value}")

# Step 2: Use the cookie to query the /countries endpoint
countries = requests.get(
    "https://country-leaders.onrender.com/countries",
    cookies=cookie_jar
)

# Step 3: Display the result
print("Countries:", countries.json())



Cookie name: user_cookie
Cookie value: c8349f6d-3802-4a63-bf24-cf852b6a526b
Countries: ['fr', 'us', 'be', 'ru', 'ma']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [5]:
# Set the leaders_url variable (1 line)

# query the /leaders endpoint, assign the output to the leaders variable (1 line)

# display the leaders variable (1 line)


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [17]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)

# display the leaders variable (1 line)
import requests

# Step 1: Get the cookie
cookie_url = "https://country-leaders.onrender.com/cookie"
response = requests.get(cookie_url)
cookie_jar = response.cookies.get_dict()

# Step 2: Set the /leaders endpoint URL with country parameter
country_code = "fr"
leaders_url = f"https://country-leaders.onrender.com/leaders?country={country_code}"

# Step 3: Query the /leaders endpoint with the cookie
leaders = requests.get(leaders_url, cookies=cookie_jar)

# Step 4: Display the result
print(leaders.json())



[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [7]:
# 4 lines


In [8]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [9]:
# < 15 lines


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [10]:
# 2 lines


## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [11]:
# 3 lines


Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [12]:
# 3 lines


That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [13]:
# 2 lines


If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [14]:
# <10 lines



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [15]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

In [16]:
# Test: 3 lines


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [17]:
# 3 lines



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [19]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [20]:
# < 20 lines

In [21]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines



Check the output of your function again.

In [23]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [24]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
# <25 lines


Test your new functions.



## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [26]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [27]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [28]:
# 3 lines


In [29]:
# Call the function (1 line)


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!